# Script para a inserção das entidades PEPs no banco


In [39]:
import pandas as pd
import os, sys, django, neomodel, dotenv, wikipedia
from datetime import datetime
import locale
locale.setlocale(locale.LC_TIME, "pt_BR.UTF-8")

BASE_DIR = os.path.abspath(os.path.join(os.getcwd(), "..", ".."))
sys.path.append(BASE_DIR)
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "tcc.settings")
django.setup()
dotenv.load_dotenv()

from myapi.models import Pessoa, Organizacao, Cargo

In [40]:
%run wiki_data.ipynb

In [41]:
def insere_pep(nome, cpf, nascimento, cnpj) -> Pessoa:
    pep = None
    try:
        pep = Pessoa.nodes.get(nome=nome, cpf=cpf)
    except neomodel.DoesNotExist:
        pep = Pessoa(nome=nome, 
                     cpf=cpf, 
                     data_nascimento=pd.to_datetime(nascimento, format='%d/%m/%Y', errors='coerce'), 
                     cnpj=cnpj).save()
        print(f'Pessoa {nome} inserida com sucesso.')
    except Exception as e:
        print(f"Erro ao inserir PEP: {e}")
    return pep


def insere_organizacao(nome, cnpj) -> Organizacao:
    org = None
    try:
        org = Organizacao.nodes.get(nome=nome)
    except neomodel.DoesNotExist:
        org = Organizacao(nome=nome, cnpj=cnpj).save()
        print(f'Organização {nome} inserida com sucesso.')
    except Exception as e:
        print(f"Erro ao inserir organização: {e}")
    return org


def insere_cargo(nome) -> Cargo:
    cargo = None
    try:
        cargo = Cargo.nodes.get(nome=nome)
    except neomodel.DoesNotExist:
        cargo = Cargo(nome=nome).save()
        print(f'Cargo {nome} inserido com sucesso.')
    except Exception as e:
        print(f"Erro ao inserir cargo: {e}")
    return cargo


def relaciona_cargo_pessoa_organizacao(cargo: Cargo, pessoa: Pessoa, org:Organizacao, papel, inicio, fim) -> None:
    try:
        pessoa.cargo.connect(cargo, {'papel': papel,
                                    'data_inicio': pd.to_datetime(inicio, format='%d/%m/%Y', errors='coerce'), 
                                    'data_fim': pd.to_datetime(fim, format='%d/%m/%Y', errors='coerce') if not pd.isna(fim) else None,
                                    'grau_precisao': 5})
        cargo.pertence.connect(org)
        print(f'Relação entre {pessoa.nome}, {cargo.nome} e {org.nome} criada com sucesso.')
    except Exception as e:
        print(f"Erro ao criar relação: {e}")
        return

def nome_contem(parcial, completo):
    palavras = parcial.lower().split()
    nome_completo = completo.lower().split()
    
    return all(p in nome_completo for p in palavras)

In [42]:
csv_path = os.path.join(BASE_DIR, 'pipelines', 'data', '202507_PEP.csv')
if os.path.exists(csv_path):
	df = pd.read_csv(csv_path, encoding='latin1', dtype=str, on_bad_lines='warn', sep=';').fillna('')
else:
	print(f"Arquivo não encontrado: {csv_path}")
df.head()

,CPF,Nome_PEP,Sigla_Função,Descrição_Função,Nível_Função,Nome_Órgão,Data_Início_Exercício,Data_Fim_Exercício,Data_Fim_Carência
0,***.531.324-**,(CUMULATIVAMENTE) MARCELO ANTÔNIO CARREIRA CAV...,CAS-1,DIR. SUPERINTENDENTE,,SUDEMA,21/12/2019,Não informada,Não informada
1,***.708.767-**,AARAO DE MOURA BRITO NETO,PREFEI,PREFEITO,,MUN. DE MANGARATIBA-RJ,01/01/2017,31/12/2020,31/12/2025
2,***.810.311-**,AAVA SANTIAGO AGUIAR,VEREAD,VEREADOR,,GOIÂNIA-GO,01/01/2021,31/12/2024,31/12/2029
3,***.333.581-**,ABADIA DE MOURA MORAES,VEREAD,VEREADOR,,MUN. DE ARAPUTANGA-MT,01/01/2017,31/12/2020,31/12/2025
4,***.265.851-**,ABADIAS SOUZA DO NASCIMENTO DAMASCENO,VEREAD,VEREADOR,,ÁGUAS LINDAS DE GOIÁS-GO,01/01/2021,31/12/2024,31/12/2029


In [ ]:
for index, row in df.iterrows():
    nome = row['Nome_PEP']
    cpf = row['CPF']
    cnpj = None
    org_nome = row['Nome_Órgão']
    org_cnpj = None
    cargo_nome = row['Descrição_Função']
    inicio = row['Data_Início_Exercício']
    fim = row['Data_Fim_Exercício']
    
    busca_wiki = wikipedia.search(query=nome,results=1)
    if len(busca_wiki) == 1 and nome_contem(busca_wiki[0], nome):
        pagina_wiki = wikipedia.page(title=nome)
        #procura da data de nascimento:
        data_nascimento = find_personal_info('Nascimento',pagina_wiki)
        if len(data_nascimento) >= 2:
            try:
                nascimento = datetime.strptime(data_nascimento[0] + ' DE ' + data_nascimento[1], "%d DE %B DE %Y")
            except Exception as e:
                print(data_nascimento)
                nascimento = None
        else: nascimento = None
    
    pessoa = insere_pep(nome, cpf, nascimento, cnpj)
    org = insere_organizacao(org_nome, org_cnpj)
    cargo = insere_cargo(cargo_nome)

    if pessoa != None and org != None and cargo != None:
        relaciona_cargo_pessoa_organizacao(cargo, pessoa, org, papel=cargo_nome, inicio=inicio, fim=fim)
        if len(busca_wiki) != 1 or not nome_contem(busca_wiki[0],nome):
            continue
        elif len(busca_wiki) ==1 and nome_contem(busca_wiki[0],nome):
            # procura e insercao dos conjuges
            conjuges = find_personal_info('Côjunge',pagina_wiki)
            if conjuges:
                for conjuge in conjuges:
                    try:
                        conjuge_pessoa = Pessoa.nodes.get(nome=conjuge)
                    except neomodel.DoesNotExist as e:
                        conjuge_pessoa = insere_pep(conjuge, None, None, None)
                    if conjuge_pessoa != None:
                        pessoa.conjuge.connect(conjuge_pessoa, {'grau_precisao': 4})
                        print(f'Conjuge de {pessoa.nome} inserido e conectado com sucesso!')
            
            # procura e insercao dos filhos
            filhos = find_personal_info('Filhos(as)',pagina_wiki)
            if filhos:
                for filho in filhos:
                    try:
                        filho_pessoa = Pessoa.nodes.get(nome=filho)
                    except neomodel.DoesNotExist as e:
                        filho_pessoa = insere_pep(filho, None, None, None)
                    if filho_pessoa != None:
                        filho_pessoa.conjuge.connect(pessoa, {'grau_precisao': 4})
                        print(f'Filho de {pessoa.nome} inserido e conectado com sucesso!')

            # procura e insercao dos progenitores
            progenitores = find_personal_info('Progenitores',pagina_wiki)
            if progenitores:
                for progenitor in progenitores:
                    try:
                        proge_pessoa = Pessoa.nodes.get(nome=progenitor)
                    except neomodel.DoesNotExist as e:
                        proge_pessoa = insere_pep(progenitor, None, None, None)
                    if proge_pessoa != None:
                        pessoa.filho.connect(proge_pessoa, {'grau_precisao': 4})
                        print(f'Progenitor de {pessoa.nome} inserido e conectado com sucesso!')

Erro ao criar relação: Attempting to inflate property 'data_fim' on <Relationship element_id='5:9ae96d80-38ac-4f87-a4bd-68ab5f89d511:1152921504606846978' nodes=(<Node element_id='4:9ae96d80-38ac-4f87-a4bd-68ab5f89d511:2' labels=frozenset() properties={}>, <Node element_id='4:9ae96d80-38ac-4f87-a4bd-68ab5f89d511:4' labels=frozenset() properties={}>) type='OCUPA' properties={'data_inicio': '2019-12-21T00:00:00', 'grau_precisao': 5, 'data_fim': 'NaT', 'papel': 'DIR. SUPERINTENDENTE'}> of class 'Relacao_Cargo': time data 'Na' does not match format '%Y-%m-%d'
Relação entre AARAO DE MOURA BRITO NETO, PREFEITO e MUN. DE MANGARATIBA-RJ criada com sucesso.
Relação entre AAVA SANTIAGO AGUIAR, VEREADOR e GOIÂNIA-GO criada com sucesso.
Relação entre ABADIA DE MOURA MORAES, VEREADOR e MUN. DE ARAPUTANGA-MT criada com sucesso.
Relação entre ABADIAS SOUZA DO NASCIMENTO DAMASCENO, VEREADOR e ÁGUAS LINDAS DE GOIÁS-GO criada com sucesso.
Relação entre ABADIO RIBEIRO DA ROCHA, VEREADOR e FIGUEIRÃO-MS cri